In [6]:
import numpy as np
import pandas as pd

# Setting the seed
np.random.seed(100)

# Defining the normal distribution parameters for CLV
mean_clv = 1000
std_dev_clv = 250
sample_size = 1000  # Let's assume we want to generate data for 1000 customers

# Generating a sample CLV data to calculate quartiles
clv_sample = np.random.normal(mean_clv, std_dev_clv, sample_size)
q1, median, q3 = np.percentile(clv_sample, [25, 50, 75])

# Display the quartile values
q1, median, q3


(823.483154256497, 993.4267139771998, 1172.6912792378725)

In [7]:
import random

# Define a function to categorize based on CLV
def categorize_clv(value, q1, median, q3):
    if value < q1:
        return "Lost Cause"
    elif q1 <= value < median:
        return "Free Riders"
    elif median <= value < q3:
        return "Vulnerable Customers"
    else:
        return "Star Customers"

# Generate the data
def generate_clv_data(num_entries, q1, median, q3):
    data = []
    for _ in range(num_entries):
        # Randomly generate CLV and other attributes
        clv_value = np.random.normal(mean_clv, std_dev_clv)
        customer_id = random.randint(1, 1000)  # Assuming customer IDs range from 1 to 1000
        predicted_customer_type = categorize_clv(clv_value, q1, median, q3)
        is_campaign1_success = bool(random.getrandbits(1))
        is_campaign2_success = bool(random.getrandbits(1))
        
        # Create a record dictionary
        record = {
            'customer_id': customer_id,
            'clv': clv_value,
            'predicted_customer_type': predicted_customer_type,
            'is_campaign1_success': is_campaign1_success,
            'is_campaign2_success': is_campaign2_success
        }
        data.append(record)
    
    return data

# Generate data for 100 entries as an example
sample_data = generate_clv_data(100, q1, median, q3)

# Convert the data to a DataFrame for better visualization
df_sample_data = pd.DataFrame(sample_data)
df_sample_data.head()  # Display the first few records


,customer_id,clv,predicted_customer_type,is_campaign1_success,is_campaign2_success
0,811,947.283162,Free Riders,True,False
1,666,830.771209,Free Riders,False,True
2,318,1033.779444,Vulnerable Customers,True,True
3,38,958.936741,Free Riders,False,True
4,406,1274.930132,Star Customers,True,True


In [11]:
import requests
import json

# The base URL for your API
api_base_url = 'http://127.0.0.1:8000'

# Function to post data to the API
def post_clv_prediction(record):
    endpoint = f"{api_base_url}/clv/"
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    # No need to include prediction_id since it should be auto-incremented by the database
    data = {
        "customer_id": record['customer_id'],
        "clv": record['clv'],
        "predicted_customer_type": record['predicted_customer_type'],
        "is_campaign1_succes": record['is_campaign1_success'],
        "is_campaign2_succes": record['is_campaign2_success']
    }
    response = requests.post(endpoint, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        print("Data inserted successfully:", response.json())
    else:
        print("Failed to insert data:", response.json())

# Example usage with one record from the sample data
for record in sample_data:
    post_clv_prediction(record)


Data inserted successfully: {'predicted_customer_type': 'Free Riders', 'prediction_id': 10001, 'is_campaign2_succes': False, 'customer_id': 811, 'clv': 947.2831615068183, 'is_campaign1_succes': True}
Data inserted successfully: {'predicted_customer_type': 'Free Riders', 'prediction_id': 10002, 'is_campaign2_succes': True, 'customer_id': 666, 'clv': 830.7712085427963, 'is_campaign1_succes': False}
Data inserted successfully: {'predicted_customer_type': 'Vulnerable Customers', 'prediction_id': 10003, 'is_campaign2_succes': True, 'customer_id': 318, 'clv': 1033.7794440618627, 'is_campaign1_succes': True}
Data inserted successfully: {'predicted_customer_type': 'Free Riders', 'prediction_id': 10004, 'is_campaign2_succes': True, 'customer_id': 38, 'clv': 958.9367414657042, 'is_campaign1_succes': False}
Data inserted successfully: {'predicted_customer_type': 'Star Customers', 'prediction_id': 10005, 'is_campaign2_succes': True, 'customer_id': 406, 'clv': 1274.9301319551537, 'is_campaign1_succ